In [48]:
import os
import googlemaps
from ppprint import ppprint
import pandas as pd
import numpy as np
import time
import sys
from dotenv import load_dotenv, find_dotenv
import streamlit as st
from datetime import datetime
import numpy as np
from pandas import json_normalize
from pprint import pprint

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Path
dotenv_path = 'C:\\Users\\kevin\\Google Drive\\My Drive\\Github\\all-api-keys\\.env'
load_dotenv(dotenv_path)

# Load api key
gmaps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(gmaps_api_key)

# Initialize empty token_list
token_list = []
# Initialize master_Df
master_df = pd.DataFrame()

def find_best_restaurants(city_name, place_type, prices_allowed=[1,2,3,4], query='', open_now_boolean=False, page_token="", master_df=master_df):
  # Automatic Parameters
  lat = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lat')
  lng = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lng')
  gmaps_json = gmaps.places(location=(lat,lng),
                            type = place_type,
                            query = query,
                            min_price = min(prices_allowed),
                            max_price = max(prices_allowed),
                            open_now = open_now_boolean,
                            page_token = page_token,
                            )

  # Dataframe-ize results
  df = json_normalize(gmaps_json)
  # display(df)

  # Handle if 0 results returns:
  if df['status'].iloc[0] == 'ZERO_RESULTS':
    raise Exception(f"You have high af standards for {city_name}. Unfortunately, there are no results matching the criteria. Is it because you have Open Now checked? Please lower minimum number of reviews or minimum rating.")

  # If df['next_page_token'] exists
    # append df['next_page_token'].value to token_list
  if 'next_page_token' in df and pd.notna(df['next_page_token'].iloc[0]):
    # Append the value of 'next_page_token' to token_list
    next_page_token = df['next_page_token'].iloc[0]
    token_list.append(next_page_token)
    # Show results in dataframe
    # Explode the results into same level row
    df2 = json_normalize(df['results'].explode())
    # display(df2)
    # Explode photos too
    df3 = json_normalize(df2['photos'].explode())
    df4 = pd.concat([df2.drop('photos', axis=1), df3], axis=1)

    # display(df4)
    # Append dataframe to master dataframe
    master_df = pd.concat([master_df, df4], axis=0).reset_index(drop=True)
    # Ensure a short delay before the next request to comply with API's "next_page_token" latency
    time.sleep(2)  # Google Maps API requires a short delay before using the next_page_token
    find_best_restaurants(city_name, place_type, prices_allowed, query, open_now_boolean, next_page_token, master_df=master_df)

  # No 'next_page_token' means its already the last page.
  else:
    # Explode the results into same level row
    df2 = json_normalize(df['results'].explode())
    # display(df2)
    # Explode photos too
    df3 = json_normalize(df2['photos'].explode())
    df4 = pd.concat([df2.drop('photos', axis=1), df3], axis=1)

    # Append dataframe to master dataframe
    master_df = pd.concat([master_df, df4], axis=0).reset_index(drop=True)
    print(len(master_df))

    # Checks
    # Check and remove any duplicates
    master_df = master_df.drop_duplicates(subset='place_id', keep='first')
    print(len(master_df))
    try:
        assert max(master_df['price_level']) <= max(prices_allowed), "Maximum price level in DataFrame does not match allowed prices."
    except AssertionError as error:
        print(error)

    return master_df


# User input Parameters
find_best_restaurants('paris', 'restaurant', [1,2,3], 'sushi', False)
# city_name = 'san jose'
# place_type = 'restaurant'
# prices_allowed = [1,2,3]
# query = 'sushi'
# open_now_boolean = True

60
60


,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,place_id,price_level,rating,reference,types,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,height,html_attributions,photo_reference,width
0,OPERATIONAL,"11 Rue de la Verrerie, 75004 Paris",https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Kiccho,ChIJt1qhRx1u5kcR_v4MulbUn7Q,2,4.40,ChIJt1qhRx1u5kcR_v4MulbUn7Q,"[meal_delivery, restaurant, food, point_of_interest, establishment]",461,48.86,2.35,48.86,2.36,48.86,2.35,False,V943+VW Paris,8FW4V943+VW,2160,"[<a href=""https://maps.google.com/maps/contrib/108508698897591607525"">Berga Vanre</a>]",ATplDJZz3cJiUf4TjxrG6DUXtm0-vgt04I_0AizXe1SrWGvrYlQh4ZNT_pvmsv6ITjNnmr98OXKMWZcwh_Xdx6C3iJnkYTzziNcm4M2o-bZ1fk84VQ9YP3frJPxCCSOD_BYX4dIgTp-ENh6D3m1lpJvzi65AcCeN39oz7Un_78wm7Q0XansA,3840
1,OPERATIONAL,"9 Rue de la Verrerie, 75004 Paris",https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Kyo,ChIJtX16SB1u5kcRmIX5D5iT6Pw,2,4.20,ChIJtX16SB1u5kcRmIX5D5iT6Pw,"[bar, restaurant, food, point_of_interest, establishment]",603,48.86,2.36,48.86,2.36,48.86,2.35,False,V944+R2 Paris,8FW4V944+R2,2252,"[<a href=""https://maps.google.com/maps/contrib/111567011102140051936"">Sergio Schweiz</a>]",ATplDJaZ7d_1ABh0x8SFrzHtvcMtAFob7v0K4SAtVJPy1poqppZcqkR2sZR1sfOvzrRKfO8HouwRthHGVu6giKryZfP02hTLb1Scnhjx3jxY8oM-kM_tkJf46NNZsIxXspd032oTuenUrjOm2gkrybL7n4fPrqdgYID1ukSuSGzQcSbwyeSC,4000
2,OPERATIONAL,"13 Rue Cloche Percé, 75004 Paris",https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Le Petit Japonais,ChIJ6yE8nwJu5kcR3Ho41FKUAWg,2,4.50,ChIJ6yE8nwJu5kcR3Ho41FKUAWg,"[meal_delivery, meal_takeaway, restaurant, food, point_of_interest, establishment]",711,48.86,2.36,48.86,2.36,48.86,2.36,False,V944+HV Paris,8FW4V944+HV,3024,"[<a href=""https://maps.google.com/maps/contrib/105853516642894124228"">Bruno Refoubelet</a>]",ATplDJbClGPyTPnuBeHVz-FfgeNKz33HgZYBHb70tpJ9hg0oXIVElfyNN--xGFVygBuOLZ0SpQmouUqTMmMG3aadRs4UrSV6PUsLaAcqKzx7mG2NZK-AP28cUCNPGOr2Ccznr--NhcZvlx_L6j3wDD45KDT1jRioSZs6wr_mac_KbdgZOMoX,4032
3,OPERATIONAL,"14 Rue Cloche Percé, 75004 Paris",https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Restaurant Minori Paris,ChIJdTa4mAJu5kcRIxOXb1bLzok,2,4.60,ChIJdTa4mAJu5kcRIxOXb1bLzok,"[meal_takeaway, restaurant, food, point_of_interest, establishment]",319,48.86,2.36,48.86,2.36,48.86,2.36,False,V944+GW Paris,8FW4V944+GW,3456,"[<a href=""https://maps.google.com/maps/contrib/106773622330623218294"">Jérôme Martins</a>]",ATplDJai3iwf6F2ilF-lz_EnMT5Z7kpmdFNduQAcIl-2oAOc2A1AAssbOPglgRRjcjLyTqfmvuVaG_Lvyl3xrUxN9NlsOYyYBz80dDNBP8_JDAKfpiFv43_b5H-IjBIFN_um99h-2ghnRbKRnqlOSkdepvt4U-opcsrDnlZlCD1fakKV-sb-,4608
4,OPERATIONAL,"31 Rue Rambuteau, 75003 Paris",https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Côté Sushi Rambuteau,ChIJH9xpnRxu5kcRwcQOCRDOaGw,2,4.00,ChIJH9xpnRxu5kcRwcQOCRDOaGw,"[meal_takeaway, restaurant, food, point_of_interest, establishment]",902,48.86,2.35,48.86,2.36,48.86,2.35,False,V963+9J Paris,8FW4V963+9J,2048,"[<a href=""https://maps.google.com/maps/contrib/118119460232969747111"">yohann boua</a>]",ATplDJZScW6PEesDhCwFw6RzjgIfcZXj4cEPoBqZJ48CXe-0zRMWJ1fnhzItDbmpFbq17CqVIodMveKdTGYoJpErJt5J598fKGThtset11meW57YYZz-kq3Lp-Z_QhXbyODKBFb7wmWv8kaCFUY_eqHN5EiZC8E7WEpRNI6TYFQUxDBUKWpH,1536
5,OPERATIONAL,"156 Rue 